In [12]:
# Ejercicio 1: Sistema de Triaje para Sala COVID y Urgencias
import random
import pandas as pd
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

class Persona:
    """Clase que representa a una persona con gripa en sala de espera."""

    def __init__(self, nombre, saturacion_oxigeno, vacunas, prueba_covid=None):
        self.nombre = nombre
        self.saturacion_oxigeno = saturacion_oxigeno
        self.vacunas = vacunas
        self.prueba_covid = prueba_covid
        self.destino = None

    def __str__(self):
        return f"Persona: {self.nombre}, Saturación: {self.saturacion_oxigeno}, Vacunas: {self.vacunas}, Destino: {self.destino}"

class SistemaTriaje:
    """Sistema que decide el destino de cada paciente según sus condiciones."""

    def __init__(self, capacidad_urgencias=10):
        self.capacidad_urgencias = capacidad_urgencias
        self.camillas_ocupadas = 0

    def evaluar_paciente(self, persona):
        """Evalúa al paciente según los criterios establecidos."""
        # Criterio 1: Saturación de oxígeno
        if persona.saturacion_oxigeno < 90:
            persona.destino = "Sala COVID"
            return

        # Criterio 2: Cantidad de vacunas
        if persona.vacunas < 2:
            persona.destino = "Sala COVID"
            return

        # Criterio 3: Prueba COVID si los anteriores no son concluyentes
        if persona.prueba_covid:
            persona.destino = "Sala COVID"
            return

        # Si no cumple los criterios anteriores, va a urgencias si hay camillas
        if self.camillas_ocupadas < self.capacidad_urgencias:
            persona.destino = "Urgencias"
            self.camillas_ocupadas += 1
        else:
            persona.destino = "En espera"

class GeneradorDatos:
    """Clase para generar datos aleatorios de personas."""

    @staticmethod
    def generar_personas(cantidad):
        """Genera una lista de personas con datos aleatorios."""
        personas = []
        nombres = [f"Paciente_{i+1}" for i in range(cantidad)]

        for nombre in nombres:
            # Generamos datos aleatorios para cada persona
            saturacion = random.randint(80, 100)
            vacunas = random.randint(0, 3)
            # Prueba COVID solo si es necesaria (probabilidad del 30%)
            prueba_covid = random.random() < 0.3 if saturacion >= 90 and vacunas >= 2 else None

            personas.append(Persona(nombre, saturacion, vacunas, prueba_covid))

        return personas

class ManejadorArchivos:
    """Clase para manejar la entrada y salida de archivos."""

    @staticmethod
    def guardar_personas(personas, archivo):
        """Guarda la lista de personas en un archivo CSV."""
        data = []
        for p in personas:
            data.append({
                'Nombre': p.nombre,
                'Saturacion_Oxigeno': p.saturacion_oxigeno,
                'Vacunas': p.vacunas,
                'Prueba_COVID': 'Positivo' if p.prueba_covid else 'No realizada' if p.prueba_covid is None else 'Negativo'
            })

        df = pd.DataFrame(data)
        df.to_csv(archivo, index=False)
        return df

    @staticmethod
    def guardar_resultados(personas, archivo):
        """Guarda los resultados del triaje en un archivo CSV."""
        data = []
        for p in personas:
            data.append({
                'Nombre': p.nombre,
                'Saturacion_Oxigeno': p.saturacion_oxigeno,
                'Vacunas': p.vacunas,
                'Prueba_COVID': 'Positivo' if p.prueba_covid else 'No realizada' if p.prueba_covid is None else 'Negativo',
                'Destino': p.destino
            })

        df = pd.DataFrame(data)
        df.to_csv(archivo, index=False)
        return df

class Visualizador:
    """Clase para visualizar los resultados."""

    @staticmethod
    def graficar_resultados(personas):
        """Genera gráficos de los resultados."""
        destinos = [p.destino for p in personas]
        conteo = {destino: destinos.count(destino) for destino in set(destinos)}

        # Gráfico de torta
        plt.figure(figsize=(10, 6))
        plt.pie(conteo.values(), labels=conteo.keys(), autopct='%1.1f%%', startangle=90)
        plt.axis('equal')
        plt.title('Distribución de Pacientes por Destino')
        plt.savefig('distribucion_pacientes.png')
        plt.close()

        # Histograma de saturación de oxígeno
        saturaciones = [p.saturacion_oxigeno for p in personas]
        plt.figure(figsize=(10, 6))
        plt.hist(saturaciones, bins=10, color='skyblue', edgecolor='black')
        plt.axvline(x=90, color='r', linestyle='--', label='Umbral COVID (90)')
        plt.xlabel('Saturación de Oxígeno')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Saturación de Oxígeno')
        plt.legend()
        plt.savefig('saturacion_oxigeno.png')
        plt.close()

def main():
    # Generar 50 personas aleatorias
    generador = GeneradorDatos()
    personas = generador.generar_personas(50)

    # Guardar datos de entrada
    manejador = ManejadorArchivos()
    df_entrada = manejador.guardar_personas(personas, 'personas_entrada.csv')
    print("Archivo de entrada generado: personas_entrada.csv")

    # Evaluar a cada persona
    sistema = SistemaTriaje()
    for persona in personas:
        sistema.evaluar_paciente(persona)

    # Guardar resultados
    df_salida = manejador.guardar_resultados(personas, 'resultados_triaje.csv')
    print("Archivo de resultados generado: resultados_triaje.csv")

    # Mostrar estadísticas
    destinos_count = df_salida['Destino'].value_counts()
    print("\nResumen de resultados:")
    for destino, count in destinos_count.items():
        print(f"{destino}: {count} personas")

    # Generar visualizaciones
    visualizador = Visualizador()
    visualizador.graficar_resultados(personas)
    print("\nGráficos generados: distribucion_pacientes.png y saturacion_oxigeno.png")

if __name__ == "__main__":
    main()

Archivo de entrada generado: personas_entrada.csv
Archivo de resultados generado: resultados_triaje.csv

Resumen de resultados:
Sala COVID: 40 personas
Urgencias: 10 personas

Gráficos generados: distribucion_pacientes.png y saturacion_oxigeno.png
